In [1]:
# Lib to get the html for a url adress
from urllib.request import urlopen
# Lib to search and get specifically datas of a html or xml files
from bs4 import BeautifulSoup
# Lib to work with dataframes
import pandas as pd
# Lib to open a browser and thus fully open an infinite scrolling page
from selenium import webdriver
# Lib to use keyboard commands
from selenium.webdriver.common.keys import Keys
# lib to search for page elements
from selenium.webdriver.common.by import By
# Lib to delay the execution
import time
# Lib to search a string by a partial string in a string list
import re

In [2]:
# list to indicate when the function of scrolling page must end
end_points = pd.Series({'netflix' : 'To and From New York', 
              'hulu' : 'The Twilight Zone', 
              'disney-plus': 'Imagination Movers', 
              'hbo-max' : 'Zapped', 
              'amazon-prime-video' : 'Zoombies'})
end_points

netflix               To and From New York
hulu                     The Twilight Zone
disney-plus             Imagination Movers
hbo-max                             Zapped
amazon-prime-video                Zoombies
dtype: object

In [3]:
parental_guidelines = ['7+', '13+', '18+', '16+', 'ALL', 'ALL_AGES', 'AGES_18_', 'G', 'NC-17', 'NR', 'NOT RATED',
      'PG', 'PG-13', 'R', 'TV-NR', 'TV-PG', 'TV-14', 'TV-G', 'TV-Y', 'TV-Y7-FV', 'TV-Y7', 'TV-MA', 'UNRATED']

In [4]:
duration_check = ['min', 'Season']

In [5]:
# Function to full scrolling of an infinite scrolling page
def scrolling_page(streaming):
    
    base = 'https://flixable.com/'
    # condition to deal with the different urls
    if streaming == 'netflix':
        ad = ''
    else:
        ad = streaming
        
    # open the browser   
    driver = webdriver.Chrome()
    driver.get(base + ad)

    # delay to load the page
    time.sleep(0.5)

    # get the body of the page
    element = driver.find_element(By.TAG_NAME, "body")
    
    # variable to check when to do break condition check
    count = 0

    # loop to full open the page
    while True:
        element.send_keys(Keys.END)
        time.sleep(0.5)
        # command to go up one time to avoid loading error
        element.send_keys(Keys.PAGE_UP)
        
        count += 1
        if count == 20:
            count = 0
            # break condition check
            if driver.find_elements(By.CLASS_NAME, 'card-title')[-1].text == end_points[streaming]:
                print('Page fully open')
                break
    
    # converting and saving the page
    soup =  BeautifulSoup(driver.page_source, 'html.parser')

    return soup

In [6]:
# Function to create a list of titles links 
def links():
    list_links = []
    for link in soup.find_all("a", href=True):
        if '/title' in link["href"]:
                list_links.append(link["href"])
    return list_links

In [ ]:
# Put in this variable which streaming service you want to scraping
streaming = 'netflix'

In [40]:
# Running the scrolling function with the streaming chosen
soup = scrolling_page(streaming)

# Running the links function
list_links = links()

# Create the list variable to place dataset of each title
cards = []

# Create a list for the ids of possible errors 
errors = []

# Create the string base for the links
base = 'https://flixable.com'

# Loop for open each link
for i in range(0, len(list_links),2):

    # Variable to store the link dataset
    card = {}
    
    url = base + list_links[i]
    try:
        response = urlopen(url)
    except:
        errors.append(i)
        continue
    
    # Read and parse the HTML link       
    html = response.read()
    html = html.decode('utf-8')
    soup =  BeautifulSoup(html, 'html.parser')
    
    # Get the title
    card['title'] = soup.find('h1', {'class' : 'title'}).getText()
    
    # Create some columns for datas
    card['release_year'] = 0
    card['parental_guidelines'] = ' '
    card['duration'] = ' '
    
    # Get the year, the parental guidelines and the duration
    size_mr2 = len(soup.findAll('span', {'class' : 'mr-2'}))
    for j in range(size_mr2):
        _ = soup.findAll('span', {'class' : 'mr-2'})[j].getText()
        if j == 0:
            card['release_year'] = _

        if _ in parental_guidelines:
            card['parental_guidelines'] = _
        
        if duration_check[0] in _ or duration_check[1] in _:
            card['duration'] = _
    
    # Get the genre
    x = str(soup.findAll('a', href=True))
    card['genre'] = []
    genres = re.findall(r'genre/(.*?)"', x)     
    if len(genres) > 0:
        for g in genres:
            if streaming == 'netflix':
                href = '/genre/' + g
            else:
                href = '/' + streaming + '/genre/' + g
            try:
                card['genre'].append(soup.find('a', href=href).getText())
            except:
                print('genre error')
                continue
    else:
        card['genre'] = ' '

    # Get the date_added
    try:
        card['date_added'] = soup.find('p', {'class' : 'mb-2'}).getText().strip().split(':')[1]
    except:
        card['date_added'] = ' '

    # Get the average_rating
    try:
        average_rating = soup.h6.contents[5].getText().split('/')[0]
        card['average_rating'] = average_rating
    except:
        card['average_rating'] = ' '

    # Get the description
    try:
        card['description'] = soup.findAll('p', {'class' : 'card-description'})[0].getText().strip()
    except:
        card['description'] = ' '

    # Join the link dataset to the list
    cards.append(card)
    
    # Completion percentage indicator
    print(round(i/len(list_links) * 100, 2), '%| indice:', i, '/', len(list_links))

Page fully open
0.0 %| indice: 0 / 12206
0.02 %| indice: 2 / 12206
0.03 %| indice: 4 / 12206
0.05 %| indice: 6 / 12206
0.07 %| indice: 8 / 12206
0.08 %| indice: 10 / 12206
0.1 %| indice: 12 / 12206
0.11 %| indice: 14 / 12206
0.13 %| indice: 16 / 12206
0.15 %| indice: 18 / 12206
0.16 %| indice: 20 / 12206
0.18 %| indice: 22 / 12206
0.2 %| indice: 24 / 12206
0.21 %| indice: 26 / 12206
0.23 %| indice: 28 / 12206
0.25 %| indice: 30 / 12206
0.26 %| indice: 32 / 12206
0.28 %| indice: 34 / 12206
0.29 %| indice: 36 / 12206
0.31 %| indice: 38 / 12206
0.33 %| indice: 40 / 12206
0.34 %| indice: 42 / 12206
0.36 %| indice: 44 / 12206
0.38 %| indice: 46 / 12206
0.39 %| indice: 48 / 12206
0.41 %| indice: 50 / 12206
0.43 %| indice: 52 / 12206
0.44 %| indice: 54 / 12206
0.46 %| indice: 56 / 12206
0.48 %| indice: 58 / 12206
0.49 %| indice: 60 / 12206
0.51 %| indice: 62 / 12206
0.52 %| indice: 64 / 12206
0.54 %| indice: 66 / 12206
0.56 %| indice: 68 / 12206
0.57 %| indice: 70 / 12206
0.59 %| indice: 72 /

4.85 %| indice: 592 / 12206
4.87 %| indice: 594 / 12206
4.88 %| indice: 596 / 12206
4.9 %| indice: 598 / 12206
4.92 %| indice: 600 / 12206
4.93 %| indice: 602 / 12206
4.95 %| indice: 604 / 12206
4.96 %| indice: 606 / 12206
4.98 %| indice: 608 / 12206
5.0 %| indice: 610 / 12206
5.01 %| indice: 612 / 12206
5.03 %| indice: 614 / 12206
5.05 %| indice: 616 / 12206
5.06 %| indice: 618 / 12206
5.08 %| indice: 620 / 12206
5.1 %| indice: 622 / 12206
5.11 %| indice: 624 / 12206
5.13 %| indice: 626 / 12206
5.15 %| indice: 628 / 12206
5.16 %| indice: 630 / 12206
5.18 %| indice: 632 / 12206
5.19 %| indice: 634 / 12206
5.21 %| indice: 636 / 12206
5.23 %| indice: 638 / 12206
5.24 %| indice: 640 / 12206
5.26 %| indice: 642 / 12206
5.28 %| indice: 644 / 12206
5.29 %| indice: 646 / 12206
5.31 %| indice: 648 / 12206
5.33 %| indice: 650 / 12206
5.34 %| indice: 652 / 12206
5.36 %| indice: 654 / 12206
5.37 %| indice: 656 / 12206
5.39 %| indice: 658 / 12206
5.41 %| indice: 660 / 12206
5.42 %| indice: 662 / 1

9.62 %| indice: 1174 / 12206
9.63 %| indice: 1176 / 12206
9.65 %| indice: 1178 / 12206
9.67 %| indice: 1180 / 12206
9.68 %| indice: 1182 / 12206
9.7 %| indice: 1184 / 12206
9.72 %| indice: 1186 / 12206
9.73 %| indice: 1188 / 12206
9.75 %| indice: 1190 / 12206
9.77 %| indice: 1192 / 12206
9.78 %| indice: 1194 / 12206
9.8 %| indice: 1196 / 12206
9.81 %| indice: 1198 / 12206
9.83 %| indice: 1200 / 12206
9.85 %| indice: 1202 / 12206
9.86 %| indice: 1204 / 12206
9.88 %| indice: 1206 / 12206
9.9 %| indice: 1208 / 12206
9.91 %| indice: 1210 / 12206
9.93 %| indice: 1212 / 12206
9.95 %| indice: 1214 / 12206
9.96 %| indice: 1216 / 12206
9.98 %| indice: 1218 / 12206
10.0 %| indice: 1220 / 12206
10.01 %| indice: 1222 / 12206
10.03 %| indice: 1224 / 12206
10.04 %| indice: 1226 / 12206
10.06 %| indice: 1228 / 12206
10.08 %| indice: 1230 / 12206
10.09 %| indice: 1232 / 12206
10.11 %| indice: 1234 / 12206
10.13 %| indice: 1236 / 12206
10.14 %| indice: 1238 / 12206
10.16 %| indice: 1240 / 12206
10.18 %

14.12 %| indice: 1724 / 12206
14.14 %| indice: 1726 / 12206
14.16 %| indice: 1728 / 12206
14.17 %| indice: 1730 / 12206
14.19 %| indice: 1732 / 12206
14.21 %| indice: 1734 / 12206
14.22 %| indice: 1736 / 12206
14.24 %| indice: 1738 / 12206
14.26 %| indice: 1740 / 12206
14.27 %| indice: 1742 / 12206
14.29 %| indice: 1744 / 12206
14.3 %| indice: 1746 / 12206
14.32 %| indice: 1748 / 12206
14.34 %| indice: 1750 / 12206
14.35 %| indice: 1752 / 12206
14.37 %| indice: 1754 / 12206
14.39 %| indice: 1756 / 12206
14.4 %| indice: 1758 / 12206
14.42 %| indice: 1760 / 12206
14.44 %| indice: 1762 / 12206
14.45 %| indice: 1764 / 12206
14.47 %| indice: 1766 / 12206
14.48 %| indice: 1768 / 12206
14.5 %| indice: 1770 / 12206
14.52 %| indice: 1772 / 12206
14.53 %| indice: 1774 / 12206
14.55 %| indice: 1776 / 12206
14.57 %| indice: 1778 / 12206
14.58 %| indice: 1780 / 12206
14.6 %| indice: 1782 / 12206
14.62 %| indice: 1784 / 12206
14.63 %| indice: 1786 / 12206
14.65 %| indice: 1788 / 12206
14.66 %| indic

18.63 %| indice: 2274 / 12206
18.65 %| indice: 2276 / 12206
18.66 %| indice: 2278 / 12206
18.68 %| indice: 2280 / 12206
18.7 %| indice: 2282 / 12206
18.71 %| indice: 2284 / 12206
18.73 %| indice: 2286 / 12206
18.74 %| indice: 2288 / 12206
18.76 %| indice: 2290 / 12206
18.78 %| indice: 2292 / 12206
18.79 %| indice: 2294 / 12206
18.81 %| indice: 2296 / 12206
18.83 %| indice: 2298 / 12206
18.84 %| indice: 2300 / 12206
18.86 %| indice: 2302 / 12206
18.88 %| indice: 2304 / 12206
18.89 %| indice: 2306 / 12206
18.91 %| indice: 2308 / 12206
18.93 %| indice: 2310 / 12206
18.94 %| indice: 2312 / 12206
18.96 %| indice: 2314 / 12206
18.97 %| indice: 2316 / 12206
18.99 %| indice: 2318 / 12206
19.01 %| indice: 2320 / 12206
19.02 %| indice: 2322 / 12206
19.04 %| indice: 2324 / 12206
19.06 %| indice: 2326 / 12206
19.07 %| indice: 2328 / 12206
19.09 %| indice: 2330 / 12206
19.11 %| indice: 2332 / 12206
19.12 %| indice: 2334 / 12206
19.14 %| indice: 2336 / 12206
19.15 %| indice: 2338 / 12206
19.17 %| in

...

81.6 %| indice: 9960 / 12206
81.62 %| indice: 9962 / 12206
81.63 %| indice: 9964 / 12206
81.65 %| indice: 9966 / 12206
81.66 %| indice: 9968 / 12206
81.68 %| indice: 9970 / 12206
81.7 %| indice: 9972 / 12206
81.71 %| indice: 9974 / 12206
81.73 %| indice: 9976 / 12206
81.75 %| indice: 9978 / 12206
81.76 %| indice: 9980 / 12206
81.78 %| indice: 9982 / 12206
81.8 %| indice: 9984 / 12206
81.81 %| indice: 9986 / 12206
81.83 %| indice: 9988 / 12206
81.84 %| indice: 9990 / 12206
81.86 %| indice: 9992 / 12206
81.88 %| indice: 9994 / 12206
81.89 %| indice: 9996 / 12206
81.91 %| indice: 9998 / 12206
81.93 %| indice: 10000 / 12206
81.94 %| indice: 10002 / 12206
81.96 %| indice: 10004 / 12206
81.98 %| indice: 10006 / 12206
81.99 %| indice: 10008 / 12206
82.01 %| indice: 10010 / 12206
82.03 %| indice: 10012 / 12206
82.04 %| indice: 10014 / 12206
82.06 %| indice: 10016 / 12206
82.07 %| indice: 10018 / 12206
82.09 %| indice: 10020 / 12206
82.11 %| indice: 10022 / 12206
82.12 %| indice: 10024 / 12206


85.96 %| indice: 10492 / 12206
85.97 %| indice: 10494 / 12206
85.99 %| indice: 10496 / 12206
86.01 %| indice: 10498 / 12206
86.02 %| indice: 10500 / 12206
86.04 %| indice: 10502 / 12206
86.06 %| indice: 10504 / 12206
86.07 %| indice: 10506 / 12206
86.09 %| indice: 10508 / 12206
86.11 %| indice: 10510 / 12206
86.12 %| indice: 10512 / 12206
86.14 %| indice: 10514 / 12206
86.15 %| indice: 10516 / 12206
86.17 %| indice: 10518 / 12206
86.19 %| indice: 10520 / 12206
86.2 %| indice: 10522 / 12206
86.22 %| indice: 10524 / 12206
86.24 %| indice: 10526 / 12206
86.25 %| indice: 10528 / 12206
86.27 %| indice: 10530 / 12206
86.29 %| indice: 10532 / 12206
86.3 %| indice: 10534 / 12206
86.32 %| indice: 10536 / 12206
86.33 %| indice: 10538 / 12206
86.35 %| indice: 10540 / 12206
86.37 %| indice: 10542 / 12206
86.38 %| indice: 10544 / 12206
86.4 %| indice: 10546 / 12206
86.42 %| indice: 10548 / 12206
86.43 %| indice: 10550 / 12206
86.45 %| indice: 10552 / 12206
86.47 %| indice: 10554 / 12206
86.48 %| in

90.32 %| indice: 11024 / 12206
90.33 %| indice: 11026 / 12206
90.35 %| indice: 11028 / 12206
90.37 %| indice: 11030 / 12206
90.38 %| indice: 11032 / 12206
90.4 %| indice: 11034 / 12206
90.41 %| indice: 11036 / 12206
90.43 %| indice: 11038 / 12206
90.45 %| indice: 11040 / 12206
90.46 %| indice: 11042 / 12206
90.48 %| indice: 11044 / 12206
90.5 %| indice: 11046 / 12206
90.51 %| indice: 11048 / 12206
90.53 %| indice: 11050 / 12206
90.55 %| indice: 11052 / 12206
90.56 %| indice: 11054 / 12206
90.58 %| indice: 11056 / 12206
90.59 %| indice: 11058 / 12206
90.61 %| indice: 11060 / 12206
90.63 %| indice: 11062 / 12206
90.64 %| indice: 11064 / 12206
90.66 %| indice: 11066 / 12206
90.68 %| indice: 11068 / 12206
90.69 %| indice: 11070 / 12206
90.71 %| indice: 11072 / 12206
90.73 %| indice: 11074 / 12206
90.74 %| indice: 11076 / 12206
90.76 %| indice: 11078 / 12206
90.78 %| indice: 11080 / 12206
90.79 %| indice: 11082 / 12206
90.81 %| indice: 11084 / 12206
90.82 %| indice: 11086 / 12206
90.84 %| i

94.67 %| indice: 11556 / 12206
94.69 %| indice: 11558 / 12206
94.71 %| indice: 11560 / 12206
94.72 %| indice: 11562 / 12206
94.74 %| indice: 11564 / 12206
94.76 %| indice: 11566 / 12206
94.77 %| indice: 11568 / 12206
94.79 %| indice: 11570 / 12206
94.81 %| indice: 11572 / 12206
94.82 %| indice: 11574 / 12206
94.84 %| indice: 11576 / 12206
94.85 %| indice: 11578 / 12206
94.87 %| indice: 11580 / 12206
94.89 %| indice: 11582 / 12206
94.9 %| indice: 11584 / 12206
94.92 %| indice: 11586 / 12206
94.94 %| indice: 11588 / 12206
94.95 %| indice: 11590 / 12206
94.97 %| indice: 11592 / 12206
94.99 %| indice: 11594 / 12206
95.0 %| indice: 11596 / 12206
95.02 %| indice: 11598 / 12206
95.04 %| indice: 11600 / 12206
95.05 %| indice: 11602 / 12206
95.07 %| indice: 11604 / 12206
95.08 %| indice: 11606 / 12206
95.1 %| indice: 11608 / 12206
95.12 %| indice: 11610 / 12206
95.13 %| indice: 11612 / 12206
95.15 %| indice: 11614 / 12206
95.17 %| indice: 11616 / 12206
95.18 %| indice: 11618 / 12206
95.2 %| ind

99.03 %| indice: 12088 / 12206
99.05 %| indice: 12090 / 12206
99.07 %| indice: 12092 / 12206
99.08 %| indice: 12094 / 12206
99.1 %| indice: 12096 / 12206
99.12 %| indice: 12098 / 12206
99.13 %| indice: 12100 / 12206
99.15 %| indice: 12102 / 12206
99.16 %| indice: 12104 / 12206
99.18 %| indice: 12106 / 12206
99.2 %| indice: 12108 / 12206
99.21 %| indice: 12110 / 12206
99.23 %| indice: 12112 / 12206
99.25 %| indice: 12114 / 12206
99.26 %| indice: 12116 / 12206
99.28 %| indice: 12118 / 12206
99.3 %| indice: 12120 / 12206
99.31 %| indice: 12122 / 12206
99.33 %| indice: 12124 / 12206
99.34 %| indice: 12126 / 12206
99.36 %| indice: 12128 / 12206
99.38 %| indice: 12130 / 12206
99.39 %| indice: 12132 / 12206
99.41 %| indice: 12134 / 12206
99.43 %| indice: 12136 / 12206
99.44 %| indice: 12138 / 12206
99.46 %| indice: 12140 / 12206
99.48 %| indice: 12142 / 12206
99.49 %| indice: 12144 / 12206
99.51 %| indice: 12146 / 12206
99.52 %| indice: 12148 / 12206
99.54 %| indice: 12150 / 12206
99.56 %| in

In [13]:
# Create dataframe and save as a .csv file
df_disney_plus = pd.DataFrame(cards)
df_disney_plus.to_csv('df_disney_plus.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [25]:
# Create dataframe and save as a .csv file
df_hulu = pd.DataFrame(cards)
df_hulu.to_csv('df_hulu.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [31]:
# Create dataframe and save as a .csv file
df_hbo_max = pd.DataFrame(cards)
df_hbo_max.to_csv('df_hbo_max.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [36]:
# Create dataframe and save as a .csv file
df_prime = pd.DataFrame(cards)
df_prime.to_csv('df_prime.csv', sep=';', index = False, encoding = 'utf-8-sig')

In [45]:
# Create dataframe and save as a .csv file
df_netflix = pd.DataFrame(cards)
df_netflix.to_csv('df_netflix.csv', sep=';', index = False, encoding = 'utf-8-sig')